### SVD Decomposition    

In [1]:
import pyJvsip as pjv
f='%.5f'

Solve using SVD Class

Create some data `A x = b`   
Here we create matrix view `A` and vector view `x` with random data and calculate vector view `b` directly using a matrix product. To demonstrate we work backward and use `A` and `b` to estimate `x`.

In [2]:
n=5
A=pjv.create('mview_d',n,n).fill(0.0)
A.randn(5)
x=pjv.create('vview_d',n).randn(9)
print('Matrix A');A.mprint(f)
print('Known x vector');x.mprint(f)
b=A.prod(x)
print('Calculated b=Ax vector');b.mprint(f)

Matrix A
[ 0.50802  0.53515  0.69864 -0.96027  0.23142;
  0.04042 -0.47661  0.20765  0.50621 -0.38285;
  0.15746  0.78115 -0.96815 -0.32034  0.79250;
  0.79172 -0.25782  0.12663  1.35454  0.25523;
 -0.19460  0.34111 -0.49602  0.17191  1.62412]

Known x vector
[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

Calculated b=Ax vector
[-2.02196  1.48038 -1.66977  2.12221 -1.26404]



Both LU and SV overwrite the input matrix; so to preserve our original matrix we use a copy. The LU (and SV) object will keep a reference to the copy (which means python will not garbage collect it). We solve this first using SVD. We don't overwrite our original vector so we can compare the estimated solution to the actual solution. SVD does not have a solver so we do it long hand.

In [3]:
u,s,v=A.copy.svd
u.mprint(f)
v.mprint(f)
s.mprint(f)
# check for zeros in s
assert s.leq(0.0).sumval == 0,'There is at least one zero singular value so no solution'
# only works in python2
# xe=v.prod(u.transview.prod(b)/s)
t=u.transview.prod(b)
xe=v.prod(pjv.div(t,s,t))
x.mprint(f)
xe.mprint(f)
print('Forbenius norm of x estimate - x is %.5e'%(xe-x).normFro)

[-0.16633 -0.51229  0.81948  0.05967  0.18652;
  0.31930  0.19799 -0.02838  0.10168  0.92071;
 -0.62185  0.03518 -0.11646 -0.71963  0.28397;
  0.16958  0.74590  0.55949 -0.26861 -0.17230;
 -0.67448  0.37516  0.03252  0.62935  0.08474]

[ 0.04232  0.14322  0.71005 -0.55408  0.40808;
 -0.44110 -0.21437  0.19419 -0.39443 -0.75246;
  0.40030 -0.23400  0.62540  0.55313 -0.29654;
  0.27958  0.87559 -0.00045 -0.04143 -0.39174;
 -0.75182  0.33485  0.25885  0.47933  0.16087]

[ 2.26877  1.89101  1.17381  0.74723  0.05827]

[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

Forbenius norm of x estimate - x is 1.54231e-14


Now we solve using LU. We start with the LU class directly.

Note that many class instance variables have a type selector (tSel) which is a dictionary which lets you select the decomposition type using the matrix type.

In [4]:
print('Example of LU.tSel: %s'%pjv.LU.tSel[A.type])
luObj = pjv.LU(pjv.LU.tSel[A.type],n)
_=luObj.lud(A.copy)
print('Solve for x using b. Done in place. Here we make a copy of b first. ')
xe = b.copy
luObj.solve(pjv.VSIP_MAT_NTRANS,xe).mprint(f)
print('Forbenius norm of x estimate - x is %.5e'%(xe-x).normFro)

Example of LU.tSel: lu_d
Solve for x using b. Done in place. Here we make a copy of b first. 
[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

Forbenius norm of x estimate - x is 1.83544e-15


Now we solve using the SV Class directly without using a pyJvsip convenience method.

In [5]:
svObj = pjv.SV(pjv.SV.tSel[A.type],A.collength,A.rowlength,'FULL','FULL')
print('Input Matrix'); A.mprint('%.3f')
print('Input b values');b.mprint('%.4f')
m=A.collength; n=A.rowlength
if m > n:
    s=A.rowview(0).empty.fill(0.0)
else:
    s=A.colview(0).empty.fill(0.0)
svObj.svd(A.copy,s)
print('Singular Values');s.mprint('%.4f')
V=pjv.create(A.type,n,n); U=pjv.create(A.type,m,m)
svObj.matV(0,n,V)
svObj.matU(0,m,U)
#xe=V.prod(U.transview.prod(b)/s)
t=U.transview.prod(b)
xe=V.prod(pjv.div(t,s,t))
print('x');x.mprint(f)
print('x estimate');xe.mprint(f)
print('Forbenius norm of x estimate - x is %.5e'%(xe-x).normFro)


Input Matrix
[ 0.508  0.535  0.699 -0.960  0.231;
  0.040 -0.477  0.208  0.506 -0.383;
  0.157  0.781 -0.968 -0.320  0.793;
  0.792 -0.258  0.127  1.355  0.255;
 -0.195  0.341 -0.496  0.172  1.624]

Input b values
[-2.0220  1.4804 -1.6698  2.1222 -1.2640]

Singular Values
[ 2.2688  1.8910  1.1738  0.7472  0.0583]

x
[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

x estimate
[ 0.39248 -1.35556 -0.24268  1.22453 -0.65029]

Forbenius norm of x estimate - x is 1.54231e-14
